## 1. Imports

In [4]:
import numpy as np
import math
import random
import networkx as nx
from networkx import components
from scipy.linalg import expm
from scipy.linalg import sqrtm
from tqdm import tqdm
import matplotlib.pyplot as plt

from alt_Frozen_Network_class import alt_Frozen_Network as alt_Frozen_Network_class
from Genetic_Class import Genetic_Evolution

## 2. Settings

In [5]:
##########################################
#number of networks I wish to consider
pop_size = 400
##########################################
##########################################
#network parameters
##########################################
# number of nodes
n = 10
# each node is joined with its m nearest neighbours in a ring topology
m = 2
# probability of edge rewiring
p = 0.1
# probability of edge creation
prob = random.random()
# exponential probability distribution used for sampling the initial population
y = random.random()
probability = -1/100 * np.log(1 - random.random() + random.random()*math.e**-100)

ts = np.linspace(0 , 1 , 10)

In [6]:
def kron(*matrices):
    result = np.array([[1]])
    for matrix in matrices:
        result = np.kron(result, matrix)
    return result

## 3. Target Setup

In [7]:

def F(n , prob):
    return nx.erdos_renyi_graph(n , prob)

##########################################
#target
##########################################
target = F(n , prob)


def adjacency_matrix_target():
    return [np.array(nx.to_numpy_array(target).astype("complex")) for _ in range(n)]

Starting state setup

In [8]:
k = 1
rho0 = np.zeros((n , 1))
rho0[k] = 1
state = kron(rho0 , np.conjugate(rho0.T))

##########################################
# Adding the sink to the target list
##########################################
def target_sink():
    mat_list = adjacency_matrix_target()
    for i in range(n):
        mat_list[i][i , i] -= 1j
    return mat_list
target_sink = target_sink()

In [9]:

#target
def evolution_target(t):
    #expmatrix = expm(complex(0 , -1) * target_sink()[i] * t)
        return [
            
                1-(np.trace
                    ((expm(complex(0 , -1) * target_sink[i] * t))
                    @ state
                    @ (np.conjugate(expm(complex(0 , -1) * target_sink[i] * t).T))
                  )
                ).real
            
            for i in range(n)
            ]
def target_data():
    data = []
    for t in ts:
        data.append(evolution_target(t))
    return data


## 4. Population Setup

In [10]:
def pop(pop_size, n):
    return [nx.to_numpy_array(F(n, random.random())).astype("complex") for _ in range(pop_size)]
population = pop(pop_size, n)


def alt_genetic_distance(pop , start_index = 0):
    dist = []
    for i in range(start_index , pop_size):
        dist.append([alt_Frozen_Network_class(pop[i] , target_data() , ts).calculate_distance() , pop[i]])
    dist = sorted(dist , key = lambda x: x[0])
    return [d[0] for d in dist] , [d[1] for d in dist]

def fit_half(pop):
    return pop[:pop_size // 2]

alt_distances = [alt_genetic_distance(population)[0]]
ordered_pop = [alt_genetic_distance(population)[1]]
found_it = []
if alt_distances[0][0] == 0:
    found_it.append(fit_half(ordered_pop[0])[0])
    print("Found fit individual")
else:
    print("Continue to Genetic Evolution")


Continue to Genetic Evolution


In [11]:
new_pop = Genetic_Evolution(fit_half(alt_genetic_distance(population)[1]) , alt_distances[0][:len(alt_distances[0])//2]).new_population()

## 5. Genetic Algorithm

In [12]:

distances = []
best_individual = []
number_of_connections = []
new_pop_set = []
best_distance = []
mean_distance = []
fittest_individual = []
connections = []
degree_distribution = []
found_it = []

In [13]:
extreme_mutation_count = 0

for j in tqdm(range(100)):

    fit_individual = []
    if j == 0:
        new_distances, new_pop = alt_genetic_distance(new_pop)
    else:
        new_distances, new_population = alt_genetic_distance(new_pop , start_index = pop_size // 4)
        # new_distances = sorted(distances[-1][:pop_size // 4] + new_distances)
        # new_pop = new_pop[:pop_size//4] + new_population

        # Combine distances and population from current and new generations
        combined = list(zip(distances[-1][:pop_size // 4] + new_distances, new_pop[:pop_size // 4] + new_population))
        # Sort by distances
        combined.sort(key=lambda x: x[0])
        # Unpack sorted distances and population
        new_distances, new_pop = zip(*combined)
        # Convert to lists (if needed)
        new_distances = list(new_distances)
        new_pop = list(new_pop)

    distances.append(new_distances)
    best_individual.append(new_pop[0])
    number_of_connections.append(np.array([np.sum(np.real(adjacency_matrix_target()[0]))/2 , np.sum(np.real(new_pop[0]))/2 , np.sum(np.real(new_pop[1]))/2 , np.sum(np.real(new_pop[2]))/2]))
    new_pop_set.append(np.array(new_pop[:pop_size // 4]))
    degree_distribution.append([degree for node , degree in sorted(nx.degree(nx.from_numpy_array(np.real(new_pop[0]))), key=lambda x: x[1], reverse=True)])
    #Calculating the best and mean distances
    best_distance = min(distances[j])
    mean_distance = np.mean(new_distances)
    fittest_individual = new_pop[0]
    connections = np.array([np.sum(np.real(adjacency_matrix_target()[0]))/2 , np.sum(np.real(new_pop[0]))/2 , np.sum(np.real(new_pop[1]))/2 , np.sum(np.real(new_pop[2]))/2 , np.sum(np.real(new_pop[3]))/2])
    degree_dist = [degree for node , degree in sorted(nx.degree(nx.from_numpy_array(np.real(new_pop[0]))), key=lambda x: x[1], reverse=True)]

    # what to do if the algorithm finds the fittest individual
    # else rerun code
    if min(distances[j]) == 0:
        fit_individual.append(new_pop[0])
        print("fit individual found")
        break
    else:
        new_pop = Genetic_Evolution(fit_half(alt_genetic_distance(new_pop)[1]) , distances[j][:len(distances[j])//2]).new_population()
    # implementing extreme mutations if population scores aren't improving
    if j > 5:
        if (new_pop_set[j-5] == new_pop_set[j]).all():
            new_pop = Genetic_Evolution(fit_half(alt_genetic_distance(new_pop)[1]) , distances[j][:len(distances[j])//2]).EXTREME_new_population()
            # if this is performed we add +1 to the extreme_mutation_count
            extreme_mutation_count += 1
            # printing when this if performed
            print("Extreme mutation performed")
            print(extreme_mutation_count)
    # the case of reaching 3 in extreme_mutation_count means we should inject a new population of individuals
    if extreme_mutation_count == 5:
        # inject a new population of individuals and keeping old unmutated parents
        new_pop = new_pop[:int(pop_size * 0.75)] + pop(pop_size//4 , n)
        # resetting the extreme_mutation_count to zero
        extreme_mutation_count = 0
        print("New population of individuals injected")

else:
    print("No fit individual found within iteration limit")

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:13<22:29, 13.63s/it]

400


  2%|▏         | 2/100 [00:26<21:28, 13.15s/it]

400


...

400


  5%|▌         | 5/100 [01:03<19:26, 12.27s/it]

400


  5%|▌         | 5/100 [02:00<38:12, 24.14s/it]


KeyboardInterrupt: 